In [ ]:
import pandas as pd
import torch
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from transformers import get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from datasets import Dataset
import numpy as np

In [ ]:
data = pd.read_csv("/kaggle/input/dataset/dataset.csv")
data.head()

In [ ]:
data['user_gender'] = data['user_gender'].map({'Male' : 0, 'Female' : 1})

In [ ]:
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
# Convert to Hugging Face Dataset format
train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)

In [ ]:
# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

In [ ]:
# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["comment"], padding="max_length", truncation=True, max_length=128)

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

In [ ]:
# Set the format for PyTorch
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'user_gender'])
val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'user_gender'])

In [ ]:
# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

In [ ]:
# Load the BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

In [ ]:
# Set up the optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=2e-5)
total_steps = len(train_loader) * 3  # 3 epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

In [ ]:
# Training loop
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

In [ ]:
best_val_accuracy = 0

for epoch in range(2):
    model.train()
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['user_gender'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Gradient clipping
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}, Average Training Loss: {avg_train_loss}")

    # Validation loop
    model.eval()
    val_preds, val_labels = [], []
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['user_gender'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=-1)

            val_preds.extend(preds.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

    val_accuracy = accuracy_score(val_labels, val_preds)
    print(f"Validation Accuracy: {val_accuracy}")

    # Save the best model to avoid overfitting
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        model.save_pretrained("/kaggle/working/bert")
        tokenizer.save_pretrained("/kaggle/working/token")

In [ ]:
from sklearn.metrics import accuracy_score

# Evaluate the model on the validation set
model.eval()
val_preds, val_labels = [], []

with torch.no_grad():
    for batch in val_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['user_gender'].to(device)

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        # Get predicted labels by finding the index with the highest logit value
        preds = torch.argmax(logits, dim=-1)

        # Store predictions and labels
        val_preds.extend(preds.cpu().numpy())
        val_labels.extend(labels.cpu().numpy())

# Calculate accuracy
val_accuracy = accuracy_score(val_labels, val_preds)
print(f"Validation Accuracy: {val_accuracy}")

Validation Accuracy: 0.7999000552326346


In [ ]:
def load_model():
    model = BertForSequenceClassification.from_pretrained("/kaggle/working/bert")
    tokenizer = BertTokenizer.from_pretrained("/kaggle/working/token")
    model.to(device)
    model.eval()
    return model, tokenizer

In [ ]:
model, tokenizer = load_model()

In [ ]:
# Predict gender function
def predict_party(comment, model, tokenizer, device):
    model.eval()
    encoding = tokenizer(comment, padding="max_length", truncation=True, max_length=128, return_tensors="pt")
    input_ids = encoding["input_ids"].to(device)
    attention_mask = encoding["attention_mask"].to(device)
    with torch.no_grad():
        outputs = model(input_ids, attention_mask)
        prediction = torch.argmax(outputs.logits, dim=1).cpu().item()
    return "Male" if prediction == 0 else "Female"

In [ ]:
# Predict gender probability function
def predict_prob(text, model, tokenizer, device, max_length=128):
    model.eval()
    encoding = tokenizer(text, return_tensors='pt', max_length=max_length, padding='max_length', truncation=True)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    with torch.no_grad():
        outputs = model(input_ids, attention_mask)
        softmax = torch.nn.Softmax(dim=1)
        probabilities = softmax(outputs.logits).cpu().numpy()
    return probabilities

In [ ]:
# Predict instance function
def predict_instance(text, model, tokenizer, device):
    preds = predict_gender_prob(text, model, tokenizer, device)

    return {
        "Male Probability": preds[0][0],
        "Female Probability": preds[0][1]
    }

In [ ]:
# Example usage
txt = "I've had the 50watter since Oct of last year and I'm still impressed and blown away every time I play it. It's unreal. Killer choice"
predict_party(txt, model, tokenizer, device)

'Male'

In [ ]:
txt = "You l've got a good man there hun. Take care of each other and it'll last a long time.\nFor sore throats my dad used to take 2 tablespoons of apple cider vinegar and the same amount of honey, mix it in at least 8oz of hot water. Drink it while ot's still hot, but not burning. Worked every time for me...still does."
predict_party(txt, model, tokenizer, device)

'Female'